# Underperformer Descriptive Plots

Two descriptive visualisations for the underperformer extension:
1. **All underperformers GDP**: Individual Δ4 log GDP growth trajectories (±16 quarters) for every matched underperformance event.
2. **Average by feature**: Mean Δ4 log growth across all underperformance events for all six national-accounts features (GDP, Private Consumption, Government Consumption, Investment, Exports, Imports).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', None)

OUTPUT_DIR = '../plots'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Plots will be saved to: {os.path.abspath(OUTPUT_DIR)}/")

In [ ]:
# Load underperformer sample (same panel as replication, with underperformer flags)
df = pd.read_csv('../results/underperformer_sample.csv')

# Feature definitions: (level_col, yoy_col, display_name)
FEATURES = [
    ('gdp',                    'gdp_yoy_log_4q',                    'GDP'),
    ('private_consumption',    'private_consumption_yoy_log_4q',    'Private Consumption'),
    ('government_consumption', 'government_consumption_yoy_log_4q', 'Government Consumption'),
    ('capital_formation',      'capital_formation_yoy_log_4q',      'Investment'),
    ('exports',                'exports_yoy_log_4q',                'Exports'),
    ('imports',                'imports_yoy_log_4q',                'Imports'),
]

df = df.sort_values(['country', 'tq'])

print(f"Loaded {len(df):,} rows, {df['country'].nunique()} countries")
print(f"ever_underperformer == 1: {df[df['ever_underperformer'] == 1]['country'].nunique()} countries")

In [ ]:
# Build underperformer event list from the panel
# underperformer == 1 marks the Q2 of the WC year for each event
events_df = df[df['underperformer'] == 1][['country', 'year', 'tq']].copy()
events_df = events_df.drop_duplicates().sort_values(['year', 'country'])

# Country name mapping for labels
COUNTRY_NAMES = {
    'ARG': 'Argentina', 'BEL': 'Belgium', 'BGR': 'Bulgaria', 'BRA': 'Brazil',
    'COL': 'Colombia', 'CZE': 'Czech Rep.', 'DEU': 'Germany', 'DNK': 'Denmark',
    'ESP': 'Spain', 'FRA': 'France', 'GBR': 'England', 'HRV': 'Croatia',
    'HUN': 'Hungary', 'ITA': 'Italy', 'MEX': 'Mexico', 'PRT': 'Portugal',
    'ROU': 'Romania', 'RUS': 'Russia', 'SWE': 'Sweden',
}

# Build tuples: (iso3, year, label)
underperformers = []
for _, row in events_df.iterrows():
    iso3 = row['country']
    yr = int(row['year'])
    label = f"{COUNTRY_NAMES.get(iso3, iso3)} {yr}"
    underperformers.append((iso3, yr, label))

print(f"Underperformance events: {len(underperformers)}")
for iso3, yr, lbl in underperformers:
    print(f"  {lbl} ({iso3})")

In [ ]:
def extract_event_series(df, events, yoy_col, window=16):
    """Extract windowed Δ4 log growth series for each event."""
    all_series = []
    for country, year, title in events:
        event_tq = year * 4 + 2
        country_df = df[df['country'] == country].copy()
        if len(country_df) == 0:
            continue
        country_df['rel_time'] = country_df['tq'] - event_tq
        event_df = country_df[
            (country_df['rel_time'] >= -window) & (country_df['rel_time'] <= window)
        ].copy()
        if len(event_df) < 20:  # skip events with too little data
            continue
        event_df['event_label'] = title
        all_series.append(event_df[['rel_time', yoy_col, 'event_label']])
    if all_series:
        return pd.concat(all_series, ignore_index=True)
    return pd.DataFrame()

## 1. All Underperformers: Individual GDP Trajectories

In [ ]:
# All underperformer events overlaid (Δ4 log GDP growth)
yoy_col = 'gdp_yoy_log_4q'
up_gdp_df = extract_event_series(df, underperformers, yoy_col)

fig, ax = plt.subplots(figsize=(14, 6))

for event_label in up_gdp_df['event_label'].unique():
    event_data = up_gdp_df[up_gdp_df['event_label'] == event_label]
    ax.plot(event_data['rel_time'], event_data[yoy_col], 'o-', alpha=0.5,
            linewidth=1.5, markersize=3, label=event_label)

ax.axvline(0, color='red', linestyle='--', linewidth=2.5, alpha=0.9, label='World Cup')
ax.axhline(0, color='gray', linestyle='-', linewidth=0.5, alpha=0.5)
ax.axvspan(0, 16, alpha=0.08, color='red')

ax.set_xlabel('Quarters relative to World Cup', fontsize=12)
ax.set_ylabel('Δ4 log GDP Growth', fontsize=12)
ax.set_title('All Underperformers: Δ4 log GDP Growth (±16 quarters)',
             fontsize=14, fontweight='bold')
ax.set_xticks(range(-16, 17, 4))
ax.legend(loc='upper left', fontsize=8, ncol=3)
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_path = os.path.join(OUTPUT_DIR, 'underperformer_all_gdp_overlay.png')
fig.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"Events plotted: {up_gdp_df['event_label'].nunique()}")
print(f"Saved: {save_path}")
plt.show()
plt.close(fig)

## 2. Average Δ4 log Growth by Feature (All Underperformers)

In [ ]:
# Combined plot: Average Δ4 log growth for all 6 features (underperformers)

fig, ax = plt.subplots(figsize=(14, 7))

colors  = ['steelblue', 'coral', 'mediumpurple', 'forestgreen', 'orange', 'crimson']
markers = ['o', 's', '^', 'D', 'v', 'P']

for i, (level_col, yoy_col_feat, feature_name) in enumerate(FEATURES):
    feat_df = extract_event_series(df, underperformers, yoy_col_feat)
    if len(feat_df) == 0:
        print(f"No data for {feature_name}")
        continue
    avg_series = feat_df.groupby('rel_time')[yoy_col_feat].mean()
    ax.plot(avg_series.index, avg_series.values, f'{markers[i]}-',
            color=colors[i], linewidth=2, markersize=5,
            label=feature_name, alpha=0.8)
    print(f"{feature_name}: {feat_df['event_label'].nunique()} events")

ax.axvline(0, color='red', linestyle='--', linewidth=2.5, alpha=0.9, label='World Cup')
ax.axhline(0, color='gray', linestyle='-', linewidth=0.5, alpha=0.5)
ax.axvspan(0, 16, alpha=0.05, color='red')

ax.set_xlabel('Quarters relative to World Cup', fontsize=12)
ax.set_ylabel('Average Δ4 log Growth', fontsize=12)
ax.set_title('World Cup Underperformers: Average Δ4 log Growth by Feature',
             fontsize=14, fontweight='bold')
ax.set_xticks(range(-16, 17, 4))
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_path = os.path.join(OUTPUT_DIR, 'underperformer_all_features_avg.png')
fig.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"\nSaved: {save_path}")
plt.show()
plt.close(fig)